In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")



In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x765dbb23f820>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x765dbb23eb60>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")])

AIMessage(content="Hi Krish, it's nice to meet you!\n\nBeing a Chief AI Engineer is a fascinating role. What kind of projects are you working on these days?  \n\nI'm always eager to learn more about the advancements happening in the field of AI.\n", response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 22, 'total_tokens': 78, 'completion_time': 0.101818182, 'prompt_time': 0.002130454, 'queue_time': 0.021176544, 'total_time': 0.103948636}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-aaf241ab-d265-4d06-b43a-9f9301687a3b-0')

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer"),
        AIMessage(content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days?\n\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Krish, and you are a Chief AI Engineer!  \n\nIs there anything else you'd like me to remember about you? Perhaps your company or a specific project you're working on?  The more information you give me, the better I can assist you. 😊 \n", response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 75, 'total_tokens': 136, 'completion_time': 0.110909091, 'prompt_time': 0.003847131, 'queue_time': 0.020503957, 'total_time': 0.114756222}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f04aa8db-59cb-4b4c-9149-01df7aead972-0')

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [6]:
print(with_message_history)

bound=RunnableBinding(bound=RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), config={'run_name': 'check_sync_or_async'}), config={'run_name': 'RunnableWithMessageHistory'}) get_session_history=<function get_session_history at 0x765dbb2b0790> history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)]


In [7]:
config={"configurable":{"session_id":"krish1"}}

In [8]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Krish and professionals love me for my teaching")],
    config=config
)
response

AIMessage(content="That's wonderful, Krish! It's fantastic that you have a talent for teaching and that professionals appreciate your skills. \n\nWhat kind of teaching do you do? Do you have a particular subject or age group that you specialize in?  \n\nI'd love to hear more about your experience!\n", response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 22, 'total_tokens': 87, 'completion_time': 0.118181818, 'prompt_time': 0.002104715, 'queue_time': 0.016421653, 'total_time': 0.120286533}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-db78277e-a27b-4dc7-bcc9-c91ed24645c5-0')

In [9]:
with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config,
)

AIMessage(content='Your name is Krish.  😊 \n\nI remember that from our earlier conversation!\n', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 100, 'total_tokens': 120, 'completion_time': 0.036363636, 'prompt_time': 0.004261, 'queue_time': 0.019451667, 'total_time': 0.040624636}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-968b4b0d-c27f-4be8-b1cf-8c2053286fd3-0')

In [10]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to know! 😊\n"

In [11]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you. 👋 \n\nWhat can I do for you today?\n"

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John, remember? 😊  \n\nIs there anything else I can help you with?\n'

### Prompt Tempate

In [13]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [14]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")]})

AIMessage(content="Hi Krish!\n\nIt's nice to meet you. How can I help you today?  \n\nI'm ready for your questions! 😊 \n\n", response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 31, 'total_tokens': 66, 'completion_time': 0.063636364, 'prompt_time': 0.002334905, 'queue_time': 0.017099794, 'total_time': 0.065971269}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-318309f0-55d1-4fec-b648-7718e4093209-0')

In [15]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [16]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Krish")],
    config=config
)

response

AIMessage(content="Hi Krish, it's nice to meet you! \n\nI'm happy to answer any questions you have to the best of my ability. Just ask away! 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 31, 'total_tokens': 71, 'completion_time': 0.072727273, 'prompt_time': 0.002321914, 'queue_time': 0.015592054999999999, 'total_time': 0.075049187}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0cd99ef2-9c03-4f10-b812-9601b6396625-0')

In [17]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Krish.  😊 \n\nI remember that from our first interaction! Is there anything else I can help you with?\n'

In [ ]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]get_session_historyompt | model

In [19]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")],"language":"Hindi"})
response.content

'नमस्ते कृष्ण!  \n\nमुझे बहुत खुशी है कि आपने मुझे अपना नाम बताया। मैं आपकी मदद करने के लिए तैयार हूँ।  क्या मैं आपकी कोई मदद कर सकता हूँ? \n'

In [20]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [21]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Krish")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते कृष्ण! \n\nमैं आपकी मदद करने के लिए यहाँ हूँ। आप मुझसे कुछ भी पूछ सकते हैं। \n\n'

In [22]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)
response.content

'आपका नाम कृष्ण है।  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [23]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=60,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/home/sandeep/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[SystemMessage(content="you're a good assistant"),
 HumanMessage(content="hi! I'm bob"),
 AIMessage(content='hi!'),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [24]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)
response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

'You said you like vanilla ice cream! 😄  Anything else I can help you with?\n'

In [25]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "whats 2 + 2"  😊  \n\n\n\nDo you want to try another one?\n'